# PRCP-1016-HeartDiseasePred
## Team ID -  PTID-CDS-NOV-25-3300
### Muhammed Sayees

### Section 1 – Project setup & data loading

In [4]:
# ---- Section 1: Setup ----
import numpy as np
import pandas as pd

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, average_precision_score
)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

DATA_DIR = Path("data")

# ---- Load values and labels ----
values = pd.read_csv(DATA_DIR / "values.csv")
labels = pd.read_csv(DATA_DIR / "labels.csv")

print("Values shape:", values.shape)
print("Labels shape:", labels.shape)
print(values.head())
print(labels.head())


Values shape: (180, 14)
Labels shape: (180, 2)
  patient_id  slope_of_peak_exercise_st_segment               thal  \
0     0z64un                                  1             normal   
1     ryoo3j                                  2             normal   
2     yt1s1x                                  1             normal   
3     l2xjde                                  1  reversible_defect   
4     oyt4ek                                  3  reversible_defect   

   resting_blood_pressure  chest_pain_type  num_major_vessels  \
0                     128                2                  0   
1                     110                3                  0   
2                     125                4                  3   
3                     152                4                  0   
4                     178                1                  0   

   fasting_blood_sugar_gt_120_mg_per_dl  resting_ekg_results  \
0                                     0                    2   
1            